# BioExplorer - Growing astrocytes
![](../../bioexplorer_neurons_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, AstrocyteDisplacementParams, Vector2, MovieMaker

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
mm = MovieMaker(be)
status = be.reset_scene()

In [ ]:
status = core.set_camera(
    orientation=[-0.07628849957958644, -0.20045627437046104, 0.07695927759210915, 0.9736912326245474],
    position=[498.2092117887349, 1771.0670483583942, 363.5995067104092],
    target=[546.0613498487652, 1757.0592811382753, 255.5450324101157],
)
status = core.set_renderer()

In [ ]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette_size = len(material_ids)
    palette = sns.color_palette(palette_name, palette_size)

    i = 0
    for _ in material_ids:
        colors.append(palette[i])
        opacities.append(1.0)
        shading_modes.append(be.SHADING_MODE_PERLIN)
        glossinesses.append(0.1)
        user_params.append(0.05)
        specular_exponents.append(5.0)
        emissions.append(0.0)
        refraction_indices.append(1.0)
        i += 1
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

In [ ]:
status = core.set_renderer(
    background_color=[1,1,1],
    current='bio_explorer',subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.gi_distance = 100.0
params.shadows = 1.0
params.soft_shadows = 1.0
params.epsilon_factor = 1000.0
params.max_bounces = 1
params.show_background = False
params.exposure = 1.0
status = core.set_renderer_params(params)
status = core.set_renderer()

### Load astrocytes

In [ ]:
from tqdm import tqdm
for i in tqdm(range(67)):
    assembly_name = 'Astrocytes'
    be.remove_assembly(assembly_name)
    vasculature_assembly = be.add_assembly(assembly_name)
    vasculature_model = be.add_astrocytes(
        assembly_name=assembly_name,
        population_name='Astrocytes',
        population_color_scheme=be.POPULATION_COLOR_SCHEME_ID,
        morphology_representation=be.MORPHOLOGY_REPRESENTATION_BEZIER,
        load_dendrites=True,
        morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
        realism_level=be.MORPHOLOGY_REALISM_LEVEL_ALL,
        sql_filter='guid=7178',
        displacement_params=AstrocyteDisplacementParams(
            soma=Vector2(0.03, 0.7), section=Vector2(0.5, 5.0),
            end_foot=Vector2(0.3, 1.0)
        ),
        max_distance_to_soma=i + 0.01
    )
    model_ids = be.get_model_ids()['ids']
    set_morphology_materials(model_ids[0], 'coolwarm')
    core.set_renderer()
    mm.create_snapshot(
        renderer='bio_explorer',
        path='/tmp',
        base_name='%05d' % i,
        size=[1024, 1024], samples_per_pixel=64)
